In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = {}
pillars = ["busi", "econ", "educ", "envi", "gove", "heal", "pers", "safe", "soci"]
for i in pillars:
    train_df = pd.read_csv("data/{0}_train.csv".format(i)).drop("Unnamed: 0", axis = 1)
    test_df = pd.read_csv("data/{0}_test.csv".format(i)).drop("Unnamed: 0", axis = 1)
    data[i] = {"test" : test_df, "train" : train_df}
    

In [3]:
score_table = pd.concat([
    data["busi"]["train"][["country","year","busi"]],
    data["econ"]["train"][["econ"]],data["educ"]["train"][["educ"]],data["envi"]["train"][["envi"]],
    data["gove"]["train"][["gove"]],data["heal"]["train"][["heal"]],data["pers"]["train"][["pers"]],
    data["safe"]["train"][["safe"]],data["soci"]["train"][["soci"]]
    ],axis=1)
score_table["prosperity"] = score_table.iloc[:,2:].mean(axis=1)
prosperity_table = score_table[["country","year","prosperity"]]
# Most prosperous country and year
prosperity_table.sort_values("prosperity", ascending=False).head(5)

,country,year,prosperity
1149,New Zealand,2014,78.914742
1000,New Zealand,2013,78.832643
576,Sweden,2010,78.657402
1147,Norway,2014,78.500937
725,Sweden,2011,78.487949


In [18]:
score_table.to_csv("data/scores.csv")

In [5]:
prosperity_table.head()

,country,year,prosperity
0,Afghanistan,2007,37.658975
1,Angola,2007,43.880780
2,Albania,2007,54.893284
3,United Arab Emirates,2007,61.111192
4,Argentina,2007,58.557573


In [6]:
prosperity_growth_table = pd.pivot_table(
    prosperity_table,
    values="prosperity",index="country",
    columns="year").diff(axis=1).iloc[:,1:]
prosperity_growth_table

year,2008,2009,2010,2011,2012,2013,2014
country,,,,,,,
Afghanistan,-0.011917,1.315978,-0.584741,-1.478095,1.768298,0.264235,-1.325559
Albania,1.183868,0.643685,0.486321,0.109806,0.148588,-0.704340,1.066572
Algeria,-0.344381,-0.118689,0.509839,-0.600491,-0.183917,0.939682,0.043788
Angola,0.673642,-0.018470,0.337132,-0.477237,-1.232876,1.160649,-1.406232
Argentina,-0.556873,1.332824,1.475325,0.736701,-0.597984,0.525355,-0.561982
...,...,...,...,...,...,...,...
Venezuela,-1.662391,-0.196615,0.334501,0.051589,0.296304,-1.944198,-0.958522
Vietnam,0.889697,-1.116987,-0.102815,-0.275439,0.089611,1.781094,0.854969
Yemen,0.051777,-0.240995,0.132125,-1.756730,1.038170,-0.044474,-0.322300


In [7]:
prosperity_growth_table.mean(axis=1).sort_values(ascending=False).head(5)

country
Togo         0.955408
Zimbabwe     0.904582
Chad         0.820001
Georgia      0.800023
Macedonia    0.745948
dtype: float64

#### Merging DFs - Train

_First we remove the year data from all the categories in each pillar and omit all the expunged Gallup data. Then, we join all the pillars together using isocode_

In [8]:
busi_train = pd.read_csv("data/busi_train.csv")
econ_train = pd.read_csv("data/econ_train.csv")
educ_train = pd.read_csv("data/educ_train.csv")
envi_train = pd.read_csv("data/envi_train.csv")
gove_train = pd.read_csv("data/gove_train.csv")
heal_train = pd.read_csv("data/heal_train.csv")
pers_train = pd.read_csv("data/pers_train.csv")
safe_train = pd.read_csv("data/safe_train.csv")
soci_train = pd.read_csv("data/soci_train.csv")

In [9]:
busi_train = busi_train.loc[:,~busi_train.columns.str.endswith('_year')]
busi_train = busi_train.drop(columns=['Unnamed: 0', 'start', 'ahead'])

econ_train = econ_train.loc[:,~econ_train.columns.str.endswith('_year')]
econ_train = econ_train.drop(columns=['Unnamed: 0', 'comfy', 'livstd'])

educ_train = educ_train.loc[:,~educ_train.columns.str.endswith('_year')]
educ_train = educ_train.drop(columns=['Unnamed: 0', 'edsys', 'kids'])

envi_train = envi_train.loc[:,~envi_train.columns.str.endswith('_year')]
envi_train = envi_train.drop(columns=['Unnamed: 0', 'preserve'])

gove_train = gove_train.loc[:,~gove_train.columns.str.endswith('_year')]
gove_train = gove_train.drop(columns=['Unnamed: 0', 'honeste', 'cfgov'])

heal_train = heal_train.loc[:,~heal_train.columns.str.endswith('_year')]
heal_train = heal_train.drop(columns=['Unnamed: 0', 'Sadness', 'Joy', 'healp', 'cityheal'])

pers_train = pers_train.loc[:,~pers_train.columns.str.endswith('_year')]
pers_train = pers_train.drop(columns=['Unnamed: 0', 'racial', 'immig', 'freedom', 'lgbx'])

safe_train = safe_train.loc[:,~safe_train.columns.str.endswith('_year')]
safe_train = safe_train.drop(columns=['Unnamed: 0', 'shelter', 'food', 'walking', 'stolen'])

soci_train = soci_train.loc[:,~soci_train.columns.str.endswith('_year')]
soci_train = soci_train.drop(columns=['Unnamed: 0', 'strange', 'polic', 'respect', 'friends', 'voice', 'finhelp', 'volunt', 'help', 'donate'])

In [10]:
temp = pd.merge(busi_train, econ_train, how="inner", on=["country", "isonum", "isocode", "year"])
temp_1 = pd.merge(temp, soci_train, how="inner", on=["country", "isonum", "isocode", "year"])
temp_2 = pd.merge(educ_train, envi_train, how="inner", on=["country", "isonum", "isocode", "year"])
temp_3 = pd.merge(gove_train, heal_train, how="inner", on=["country", "isonum", "isocode", "year"])
temp_4 = pd.merge(pers_train, safe_train, how="inner", on=["country", "isonum", "isocode", "year"])

temp_5 = pd.merge(temp_1, temp_2, how="inner", on=["country", "isonum", "isocode", "year"])
temp_6 = pd.merge(temp_3, temp_4, how="inner", on=["country", "isonum", "isocode", "year"])

merged_train = pd.merge(temp_5, temp_6, how="inner", on=["country", "isonum", "isocode", "year"])

In [11]:
merged_train.to_csv("data/merged_train.csv")

#### Merging DFs - Test
_Below we follow the same procedures mentioned in train_

In [12]:
busi_test = pd.read_csv("data/busi_test.csv")
econ_test = pd.read_csv("data/econ_test.csv")
educ_test = pd.read_csv("data/educ_test.csv")
envi_test = pd.read_csv("data/envi_test.csv")
gove_test = pd.read_csv("data/gove_test.csv")
heal_test = pd.read_csv("data/heal_test.csv")
pers_test = pd.read_csv("data/pers_test.csv")
safe_test = pd.read_csv("data/safe_test.csv")
soci_test = pd.read_csv("data/soci_test.csv")

In [13]:
busi_test = busi_test.loc[:,~busi_test.columns.str.endswith('_year')]
busi_test = busi_test.drop(columns=['Unnamed: 0', 'start', 'ahead'])

econ_test = econ_test.loc[:,~econ_test.columns.str.endswith('_year')]
econ_test = econ_test.drop(columns=['Unnamed: 0', 'comfy', 'livstd'])

educ_test = educ_test.loc[:,~educ_test.columns.str.endswith('_year')]
educ_test = educ_test.drop(columns=['Unnamed: 0', 'edsys', 'kids'])

envi_test = envi_test.loc[:,~envi_test.columns.str.endswith('_year')]
envi_test = envi_test.drop(columns=['Unnamed: 0', 'preserve'])

gove_test = gove_test.loc[:,~gove_test.columns.str.endswith('_year')]
gove_test = gove_test.drop(columns=['Unnamed: 0', 'honeste', 'cfgov'])

heal_test = heal_test.loc[:,~heal_test.columns.str.endswith('_year')]
heal_test = heal_test.drop(columns=['Unnamed: 0', 'Sadness', 'Joy', 'healp', 'cityheal'])

pers_test = pers_test.loc[:,~pers_test.columns.str.endswith('_year')]
pers_test = pers_test.drop(columns=['Unnamed: 0', 'racial', 'immig', 'freedom', 'lgbx'])

safe_test = safe_test.loc[:,~safe_test.columns.str.endswith('_year')]
safe_test = safe_test.drop(columns=['Unnamed: 0', 'shelter', 'food', 'walking', 'stolen'])

soci_test = soci_test.loc[:,~soci_test.columns.str.endswith('_year')]
soci_test = soci_test.drop(columns=['Unnamed: 0', 'strange', 'polic', 'respect', 'friends', 'voice', 'finhelp', 'volunt', 'help', 'donate'])

In [14]:
busi_test

,country,isonum,isocode,year,logis,bband,redu,hifi,affs,ippr,insdtf,creddtf,elcostlnl,stardtf
0,Afghanistan,4,AFG,2015,2.069573,0.004795,26.472107,3.633919,3.852955,2.457773,23.62,45.0,8.151823,93.05
1,Angola,24,AGO,2015,2.542980,0.412784,31.007937,2.593719,3.236488,2.257915,0.00,5.0,6.421622,76.79
2,Albania,8,ALB,2015,2.770000,6.573716,20.825397,3.888581,3.737438,2.896221,63.42,65.0,6.197258,90.09
3,United Arab Emirates,784,ARE,2015,3.539098,11.558400,4.285714,5.053419,5.424749,5.475348,43.74,45.0,3.157000,89.98
4,Argentina,32,ARG,2015,2.986475,15.573113,30.333333,2.583314,2.923875,2.424609,42.87,50.0,3.214868,73.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,Vietnam,704,VNM,2016,3.154763,6.483695,24.555556,3.913509,3.605158,3.050503,35.83,70.0,7.187355,81.25
294,Yemen,887,YEM,2016,2.183950,1.361715,27.396825,3.848516,2.400492,2.314462,28.08,0.0,8.184262,74.22
295,South Africa,710,ZAF,2016,3.431244,3.211005,9.333333,2.079679,5.310756,5.300020,64.29,60.0,6.508024,81.18
296,Zambia,894,ZMB,2016,2.462674,0.141622,50.555556,4.544003,3.856999,4.024301,38.96,75.0,6.467388,86.69


In [15]:
temp = pd.merge(busi_test, econ_test, how="inner", on=["country", "isonum", "isocode", "year"])
temp_1 = pd.merge(temp, soci_test, how="inner", on=["country", "isonum", "isocode", "year"])
temp_2 = pd.merge(educ_test, envi_test, how="inner", on=["country", "isonum", "isocode", "year"])
temp_3 = pd.merge(gove_test, heal_test, how="inner", on=["country", "isonum", "isocode", "year"])
temp_4 = pd.merge(pers_test, safe_test, how="inner", on=["country", "isonum", "isocode", "year"])

temp_5 = pd.merge(temp_1, temp_2, how="inner", on=["country", "isonum", "isocode", "year"])
temp_6 = pd.merge(temp_3, temp_4, how="inner", on=["country", "isonum", "isocode", "year"])

merged_test = pd.merge(temp_5, temp_6, how="inner", on=["country", "isonum", "isocode", "year"])

In [16]:
merged_test.to_csv("data/merged_test.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=da4f784b-a8aa-48e9-8797-5366ef739e9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>